In [1]:
import numpy as np
import matplotlib.pyplot as plt

# 7. 합성곱 신경망(CNN)

특히 이미지 분야에서 많이 쓰인다. 

합성곱은 공학/물리학에서 쓰이는 개념으로, "두 함수 중 하나를 반전(reverse), 이동(shift) 시켜가며 나머지 함수와의 곱을 연이어 적분하는 것"이라는 뜻. 

## 7.1 전체 구조

지금처럼 계층을 레고처럼 쌓아서 만드는 것은 동일하지만 합성곱 계층(convolutional layer)과 풀링 계층(pooling layer)이 추가된다. 

지금까지의 신경망은 인접하는 계층의 모든 뉴런과 결합되어 있었다. (완전 연결, fully connected) --> Affine 계층이다. 

이 Affine-활성화함수 세트의 계층을 여러 개 쌓는다. (마지막은 최종 결과를 위해 Softmax)

<img src="../deep_learning_images/fig_7-1.png" width=600>

반면 CNN의 구조는 

<img src="../deep_learning_images/fig_7-2.png" width=600>

와 같다. (Pooling 계층은 생략하기도 한다.) 

마지막 단에 주목해보자. 출력에 가까운 층에선 그냥 지금 여태 한 것처럼 Affine-ReLU 구성을 사용할 수 있다. 끝 단에서도 Affine-Softmax를 쓸 수 있고. 

## 7.2 합성곱 계층

CNN은 계층 사이에 3차원 데이터와 같은 입체적 데이터가 흐른다는 점에서 일반 fully connected layer와 다르다. 

### 7.2.1 완전연결 계층의 문제점 

Fully connected layer (Affine layer)의 문제점은 데이터의 shape가 무시된다는 점이다. 

가령 이미지의 경우 세로/가로/채널(색상) 으로 이뤄져 있는데, 완전연결 계층에서는 1차원으로 flatten 해주게 된다. 

하지만 공간 정보를 지워버리기 때문에 공간적으로 가까운 pixel의 값이 유사하거나 RGB 각 채널의 관계 등의 패턴 정보를 무시하게 된다. 반면 CNN은 shape를 유지한다. 

CNN에서 Convolutional layer의 입출력 데이터를 feature map이라 부른다. 입력 데이터를 input feature map, 출력 데이터를 output feature map이라 부른다. 

### 7.2.2 합성곱 연산

합성곱 연산 = (이미지 처리의) Filter(커널) 연산 

filter의 window를 이동해가며 입력데이터에 적용. 

입력과 필터에 대응하는 원소끼리 곱하고 그 총합을 구한다. (단일 곱셈-누산, fused multiply-add, FMA) 

그 과정은 다음과 같다. 

<img src="../deep_learning_images/fig_7-4.png" width=500>

첫 번째 것만 설명하자면 1*2 + 2*0 + 3*1 + 0*0 + 1*1 + 2*2 + 3*1 + 0*0 + 1*2 = 15이다. 

이런 식으로 축소된다. 

CNN에서는 필터의 매개변수가 그 동안의 '가중치'에 해당한다. 다음과 같이 편향(1*1)까지도 더해줄 수 있다. 

<img src="../deep_learning_images/fig_7-5.png" width=500>

### 7.2.3 패딩

합성곱 연산 수행 이전에 입력 데이터 주변을 특정 값(예컨데 0)으로 채우기도 한다. 이를 padding이라 한다. 

<img src="../deep_learning_images/fig_7-6.png" width=500>

위와 같이 패딩은 주로 출력크기를 조정할 목적으로 사용된다. 패딩 없이 filter 연산을 하면 크기가 줄어드는데, 여러 번 반복해야 하는 Deep CNN에선 문제가 될 수 있기 때문이다. 

위에선 패딩을 적용함으로써 입력과 출력의 크기가 같아졌다. 

패딩을 크게하면 출력이 커진다. 

### 7.2.4 스트라이드 

필터 적용하는 위치의 간격을 stride라고 한다. window의 이동 step을 정하는 것이다. 

<img src="../deep_learning_images/fig_7-7.png" width=500>

stride를 키우면 출력이 작아진다. 

<br>

패딩과 stride, 입력/필터의 크기의 관계를 수식화해보자. 

- 입력 크기를 (H,W)
- 필터 크기를 (FH, FW)
- 출력 크기를 (OH, OW)
- Padding을 P
- Stride를 S

라고 하면 

출력 크기는 아래와 같아진다. 

![e7.1](../deep_learning_images/e_7.1.png "e7.1")

이는 꼭 정수로 나눠떨어지는 값이여야 한다. (칸의 수니까) 

딥러닝 프레임워크 중에선 정수가 아닐 경우 가장 가까운 정수로 반올림하는 등 구현해놓은 경우도 많다. 

더 자세한 내용은 여기를 참고하자: http://cs231n.github.io/convolutional-networks/

### 7.2.5 3차원 데이터의 합성곱 연산

지금까지는 2차원 shape를 봤다. 3차원, 그리고 그 이상은 어떻게 될까?

차원(채널)의 갯수만큼 filter도 늘어나야 한다. 그리고 하나의 출력을 얻게 된다. 

<img src="../deep_learning_images/fig_7-9.png" width=500>

### 7.2.6 블록으로 생각하기. 

3차원 합성곱 연산은 데이터와 필터가 직육면체 블록이라 생각하면 좋다. 

앞으로 (Channel, Height, Width) 순으로 표현한다. 

이제 그럼 출력으로 직사각형 1장이 아닌 직육면체를 내보내려면 어떻게 해야할까? (다수의 채널 출력) 

필터(가중치)를 많이 사용하면 된다. 

<img src="../deep_learning_images/fig_7-10.png" width=500>

<img src="../deep_learning_images/fig_7-11.png" width=500>

FN개를 사용하면 출력 채널도 FN개가 된다. 출력의 (FN, OH, OW)를 다음 계층으로 넘기는 것이 CNN의 처리흐름이다. 

7-11에서 볼 수 있듯 필터는 (출력채널 수, 입력채널 수, 높이, 너비)의 4차원 데이터가 되어야 한다. 

이 과정을 마치고 편향(FN, 1, 1)도 더할 수 있다. (numpy broadcast로 쉽게 가능)

<img src="../deep_learning_images/fig_7-12.png" width=500>

### 7.2.7 배치 처리 

CNN에서도 입력층을 1개 차원 늘려 배치 처리가 가능하다. 

<img src="../deep_learning_images/fig_7-13.png" width=500>

배치 처리는 for문으로 N회분 처리 할 것을 한 번의 커다란 연산으로 하는 것에 지나지 않는다는 것을 기억하자. (더 효율적이기 때문)

## 7.3 풀링 계층

Conv-Relu-Pooling이 한 단위이다. 이제 Conv를 봤으니 Pooling을 봐보자. 

Pooling은 세로/가로 방향의 공간을 줄이는 연산이다. 

풀링은 여러 가지 종류가 있지만(평균 풀링 등) 대표적인 Max Pooling을 살펴보면, stride=2로 처리했을 때 window가 돌아가며 window 내의 최대 값을 뽑아낸다. 

<img src="../deep_learning_images/fig_7-14.png" width=500>

이미지 인식 분야에선 주로 최대 풀링을 사용한다. 

<h1 style='color'>stride는 2차원이어야 하는 것 아닌가? 가로로 이동하는건 그렇다 치고 세로로 이동하는건? </h1>

### 7.3.1 풀링 계층의 특징

- 학습해야 할 매개변수가 없다. 간단한 처리일 뿐이다. 
- 채널 수가 변하지 않는다. 입력 데이터 채널 = 출력 데이터 채널. 
- 입력의 변화에 영향을 적게 받는다. (Robust하다.) 데이터의 변화를 풀링이 흡수해줄 수 있기 때문이다. 

<img src="../deep_learning_images/fig_7-16.png" width=500>

## 7.4 합성곱/풀링 계층 구현하기. 

트릭을 사용하면 쉽게 구현 가능하다. 

### 7.4.1 4차원 배열

입력 데이터는 4차원이어야 한다. (배치 포함하니까) 

In [4]:
x = np.random.rand(10, 1, 28, 28) # 10개의 채널 1개짜리 28*28 데이터이다. 
x.shape

# x[0] # x의 첫 번째 데이터. print 길어지니까 주석처리함. 
# x[0,0] # == x[0][0] 첫 데이터의 첫 채널 데이터

(10, 1, 28, 28)

### 7.4.2 im2col로 데이터 전개하기. 

아까 말한 트릭이다. 합성곱 연산을 구현하려면 다중 for문을 계속 써야한다. 힘드니까 im2col이라는 함수를 쓴다. 

im2col(image to column)은 입력 데이터를 필터링하기 좋게 전개하는(펼치는) 함수이다. 

<img src="../deep_learning_images/fig_7-17.png" width=500>

<img src="../deep_learning_images/fig_7-18.png" width=500>

그림에선 그림을 이쁘게 하기 위해 stride를 크게 잡아 필터 적용 영역이 겹치지 않게 한 것이다. 실제론 대부분 겹치고, 그러면 im2col 전개 후 원소 수가 원래 블록 원소 수보다 많아진다. 

때문에 메모리를 더 소비하는 단점이 있다. 하지만 대부분의 라이브러리는 대행렬 계산에 최적화되어있기 때문에 메모리에 올리기만 하면 더 효율적으로 연산할 수 있다. 

필터도 똑같이 전개하고, 모든 것을 행렬계산한다. 

<img src="../deep_learning_images/fig_7-19.png" width=500>

im2col의 출력은 2차원 행렬이 되므로, 이를 처음과 같이 4차원으로 reshape 해준다. (다음 CNN 계층에 넣기 위해)

### 7.4.3 합성곱 계층 구현하기. 

imcol(input_data, filter_h, filter_w, stride=1, pad=0)
- input data: (데이터 수, 채널 수, 높이, 너비) 의 4차원 입력 데이터
- filter_h: 필터 높이
- filter_w: 필터 너비
- stride: stride
- pad: padding

In [5]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1,3,7,7) # (N, C, H, W)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


In [6]:
x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


In [8]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0): # 가중치(필터), 편향, stride, padding 받아 초기화. 
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad) # 입력데이터 전개
        col_W = self.W.reshape(FN, -1).T # 필터 전개
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2) 
        # reshape의 -1은 다차원 배열의 원소 수가 변환 후에도 똑같이 유지되도록 적당히 묶어줌. 
        # transpose는 다차원 배열의 축 순서를 바꿔줌. 
        
        return out

<img src="../deep_learning_images/fig_7-20.png" width=500>

여기까지가 forward 구현이다. backward 구현은 책에서 생략했다. 

주의할 것은, 역전파에선 im2col을 역으로 처리하는 col2im을 사용해야 한다. 

역전파 구현은 ../common/layer.py에 되어있다. 

### 7.4.4 풀링계층 구현하기. 

똑같이 im2col을 쓰지만, channel이 독립적이라는 점에서 다르다. 

<img src="../deep_learning_images/fig_7-21.png" width=500>

위와 같이 전개 후 전개 행렬에서 행별 최댓값을 구하고 적절한 형상으로 성형한다. 

<img src="../deep_learning_images/fig_7-22.png" width=500>

이를 구현해보자. 

In [9]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # 1. 입력데이터 전개 
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        
        # 2. 행별 최댓값 구하기
        out = np.max(col, axis=1) # 지정한 축마다 최댓값을 구할 수 있음. (Pandas df처럼 2차원만 있는게 아니니까.)
        
        # 3. 적절한 모양으로 성형 
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

마찬가지로 backward는 생략되어있다. (../common/layer.py에 구현되어있음.) 

## 7.5 CNN 구현하기. 

이 계층들을 조합하여 MNIST 데이터에 적용해보자. 

<img src="../deep_learning_images/fig_7-23.png" width=500>

위의 네트워크를 SimpleConvNet 이라는 class로 구현해보자. 

__init__(self, input_dim=(1,28,28), conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1}, hidden_size=100, output_size=10, weight_init_std=0.01)
- input_dim: 입력 데이터 (C, H, W)
- conv_param: CNN 필터 수, 크기, stride, padding
- hidden_size: 완전 연결 은닉층 뉴런 수 
- output_size: 완전 연결 출력층 뉴런 수 
- weight_init_std: 초기화 때의 가중치 표준편차 

In [10]:
class SimpleConvNet:
    def __init__(self, input_dim=(1,28,28), conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1}, hidden_size=100, output_size=10, weight_init_std=0.01):
        # 기본 변수 지정 및 conv_output_size, pool_output_size 공식에 넣어 계산. 
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad / filter_stride + 1)
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        # 가중치 매개변수들 초기화하여 self.params에 담음. 
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        # CNN을 구성하는 계층들을 생성. 신경망은 계층들을 레고처럼 쌓는 것이니까. 
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.last_layer = SoftmaxWithLoss() # 기억 안나면 5.6.3 Softmax-with-Loss 단원 참조. 
        
        # 여기까지가 초기화다. 
        
    def predict(self, x): # x를 모든 레이어에 forward 시킴. (last layer 직전까지.)
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t) # L이 나온다. Softmax --> (가령) Cross Entropy Error 계층을 붙인 결과니까. 
    
    def gradient(self, x, t): # 위의 .loss()까지의 함수를 가지고 오차역전파법으로 기울기를 구함. 
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1 # 최초의 Loss를 1로 둠. 
        dout = self.last_layer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        # 결과저장
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        gards['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine2'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        
        return grads

이를 활용하면 앞서 본 MNIST 데이터의 정확도를 99%까지 올릴 수 있다. 

## 7.6 CNN 시각화하기. 

CNN의 합성곱 계층은 입력으로 받은 이미지 데이터에서 '무엇을 보고 있는'걸까? 

### 7.6.1 1번째 층의 가중치 시각화하기. 

visualize_filter.py의 코드를 참조. 

<img src="../deep_learning_images/fig_7-24.png" width=500>

학점 전 데이터는 무작위적이지만 학습 이후엔 흑백에 규칙성이 보인다. 

이 학습 후의 이미지들은 색상이 바뀐 경계선의 edge와 국소적으로 덩어리진 영역인 blob을 보고있다. 

<img src="../deep_learning_images/fig_7-25.png" width=500>

위의 그림에서 필터를 거치면 반응한 곳에 흰 픽셀이 많이 보이는 것을 확인 가능하다. 

이처럼 합성곱 계층의 필터는 에지나 블롭 등의 원시적인 정보를 추출할 수 있다. 

이런 원시적 정보가 뒷단 계층에 전달된다는 것이 앞에서 구현한 CNN에서 일어나는 일이다. 

### 7.6.2 층 깊이에 따른 추출 정보 변화 

1번 층에선 엣지와 블롭 등의 low level 정보가 추출된다면 그 다음 층에선 뭐가 추출될까? 

딥러닝 시각화 연구에 따르면 계층이 깊어질수록 추출되는 정보(정확히는 강하게 반응하는 뉴런)는 더 추상화된다. 

AlexNet의 예시를 보면 (아래에서 자세히 설명함.)

<img src="../deep_learning_images/fig_7-26.png" width=500>

이는 처음엔 단순한 엣지에, 그 다음은 텍스터에, 그 다음은 사물의 일부에 반응하는 것을 보여주며 Deep 해질 수록 점점 사물의 '의미'를 이해하도록 변화한다고 볼 수 있다. 

## 7.7 대표적인 CNN

### 7.7.1 LeNet 

<img src="../deep_learning_images/fig_7-27.png" width=500>

손글씨 숫자 인식 네트워크. 1998년에 제안됨. 

직CNN과의 차이는 

1. 활성화 함수로 ReLU 대신 Sigmoid를 사용
2. max pooling 대신 subsampling을 하여 점점 크기가 작아짐. 

CNN의 시초라고 할 수 있다. 

### 7.7.2 AlexNet

2012년에 발표됨. 

LeNet과 비교하여 

1. 활성화 함수로 ReLU 사용
2. LRN(Local Response Normalization, 국소적 정규화) 계층 사용 
3. 드롭아웃 사용 

이 다른다. 이렇게 구조는 LeNet과 크게 다르진 않았지만 이제와서 뜬 것은 병렬 계산에 특화된 고속 연산 GPU의 보급과 빅데이터의 접근성 확대를 들 수 있다. 

## 7.8 정리

- CNN은 지금까지의 완전연결 계층 네트워크에 합성곱 계층과 풀링 계층을 새로 추가한다. 
- 합성곱 계층과 풀링 계층은 im2col(이미지를 행렬로 전개)을 이용하면 간단하고 효율적으로 구현할 수 있다. 
- CNN을 시각화하여 보면 계층이 깊어질수록 고급 정보가 추출되는 모습을 확인할 수 있다. 
- 대표적인 CNN에는 